# Project - Week 1: Cyber Monday

## Setup

In [46]:
!minikube start

😄  minikube v1.31.2 on Ubuntu 20.04 (docker/amd64)
✨  Using the docker driver based on existing profile
👍  Starting control plane node minikube in cluster minikube
🚜  Pulling base image ...
🔄  Restarting existing docker container for "minikube" ...
🐳  Preparing Kubernetes v1.27.4 on Docker 24.0.4 ...
🔗  Configuring bridge CNI (Container Networking Interface) ...
🔎  Verifying Kubernetes components...
    ▪ Using image gcr.io/k8s-minikube/storage-provisioner:v5
    ▪ Using image docker.io/kubernetesui/dashboard:v2.7.0
    ▪ Using image registry.k8s.io/metrics-server/metrics-server:v0.6.4
    ▪ Using image docker.io/kubernetesui/metrics-scraper:v1.0.8
💡  Some dashboard features require the metrics-server addon. To enable all features please run:

	minikube addons enable metrics-server	


🌟  Enabled addons: storage-provisioner, default-storageclass, metrics-server, dashboard
🏄  Done! kubectl is now configured to use "minikube" cluster and "default" namespace by default


## Explore

### Create Pod

Let us start by creating a simple nginx pod. We start by writing a config file with the specifications of the pod.

In [47]:
%%writefile nginx-pod.yml
apiVersion: v1
kind: Pod
metadata:
  name: nginx-pod
  labels:
    app: just-a-pod
spec:
  containers:
    - name: nginx-pod
      image: nginx:latest
      ports:
      - containerPort: 80

Overwriting nginx-pod.yml


We can create the pod using `kubectl apply`

In [48]:
!kubectl apply -f nginx-pod.yml

pod/nginx-pod created


We can verify that our pod has been successfully created using `kubectl get pods`

In [50]:
!kubectl get pods

NAME        READY   STATUS    RESTARTS   AGE
nginx-pod   1/1     Running   0          7s


### Apply Deployment

Now we are going to take that same pod but wrap it in a deployment so that 3 Replicas of that Pod exist at any given time. Please refer to K8s Deployment documentation as needed. Complete the YAML file to:


a. Add 3 replicas

b. Add a selector that matches labels just-a-pod

c. Add a template for the nginx container used in nginx-pod.yml

In [51]:
%%writefile nginx-deploy.yml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-deployment
spec:
  replicas: 3
  selector:
    matchLabels:
      app: just-a-pod
  template:
    metadata:
      labels:
        app: just-a-pod
    spec:
      containers:
        - name: nginx-pod
          image: nginx:latest
          ports:
            - containerPort: 80

Overwriting nginx-deploy.yml


In [53]:
%%bash
sleep 2
kubectl apply -f nginx-deploy.yml
kubectl get pods

deployment.apps/nginx-deployment unchanged
NAME                                READY   STATUS    RESTARTS   AGE
nginx-deployment-59d4df587f-6w5c5   1/1     Running   0          16s
nginx-deployment-59d4df587f-gvxvl   1/1     Running   0          16s
nginx-deployment-59d4df587f-m8b7l   1/1     Running   0          16s
nginx-pod                           1/1     Running   0          42s


### Execute Logging

As users start pinging these pods (which won't happen yet fortunately for us 😊), we want to collect metadata around what items they are purchasing for Cyber Monday. Since our logging and metrics aren't set in place yet, we are going to try to collect this metadata by supporting an automated script to take backups of a logfile within our pods and push it into our local machine. To do so we will:


  a. Copy our logging script `log.sh` into our pod, executing it, and observing it write metadata to `/tmp/foo.log` to simulate users pinging the pods and data getting logged.

  b. Run a kubectl command to copy the `/tmp/foo.log` file to our local machine's working directory /workspaces/intro-to-kube/cyber-monday

In [54]:
%%bash
kubectl cp log.sh nginx-pod:/tmp/log.sh 2>/dev/null
kubectl exec -t nginx-pod -- sh -c "ls -la /tmp" 2>/dev/null

total 12
drwxrwxrwt 1 root root 4096 Sep 25 04:22 .
drwxr-xr-x 1 root root 4096 Sep 25 04:21 ..
-rw-rw-rw- 1 1000 root 3085 Sep 25 04:22 log.sh


In [55]:
%%bash
kubectl exec -t nginx-pod -- bash /tmp/log.sh 2>/dev/null
kubectl exec -t nginx-pod -- sh -c "tail -n 10 /tmp/foo.log" 2>/dev/null

{"item": "power bank", "user": 162, "date": "2023-09-25T04:22:17Z"}
{"item": "microcontroller", "user": 242, "date": "2023-09-25T04:22:17Z"}
{"item": "USB microscope", "user": 795, "date": "2023-09-25T04:22:17Z"}
{"item": "e-reader", "user": 681, "date": "2023-09-25T04:22:17Z"}
{"item": "robotic prosthetic", "user": 817, "date": "2023-09-25T04:22:17Z"}
{"item": "wireless barcode scanner", "user": 619, "date": "2023-09-25T04:22:17Z"}
{"item": "home theater projector", "user": 695, "date": "2023-09-25T04:22:17Z"}
{"item": "graphics card", "user": 317, "date": "2023-09-25T04:22:17Z"}
{"item": "e-ink display", "user": 35, "date": "2023-09-25T04:22:17Z"}
{"item": "game controller", "user": 295, "date": "2023-09-25T04:22:17Z"}


In [56]:
%%bash
kubectl cp nginx-pod:/tmp/foo.log foo.log 2>/dev/null
cat ./foo.log | tail -n 10

tar: Removing leading `/' from member names
{"item": "power bank", "user": 162, "date": "2023-09-25T04:22:17Z"}
{"item": "microcontroller", "user": 242, "date": "2023-09-25T04:22:17Z"}
{"item": "USB microscope", "user": 795, "date": "2023-09-25T04:22:17Z"}
{"item": "e-reader", "user": 681, "date": "2023-09-25T04:22:17Z"}
{"item": "robotic prosthetic", "user": 817, "date": "2023-09-25T04:22:17Z"}
{"item": "wireless barcode scanner", "user": 619, "date": "2023-09-25T04:22:17Z"}
{"item": "home theater projector", "user": 695, "date": "2023-09-25T04:22:17Z"}
{"item": "graphics card", "user": 317, "date": "2023-09-25T04:22:17Z"}
{"item": "e-ink display", "user": 35, "date": "2023-09-25T04:22:17Z"}
{"item": "game controller", "user": 295, "date": "2023-09-25T04:22:17Z"}


### Execute Logging in Pods

In [57]:
%%writefile autologging.sh
#!/bin/bash
DEPLOYED_PODS=$(kubectl get pods | awk '{print $1}' | grep 'deployment')
for pod in $DEPLOYED_PODS
do
  kubectl cp log.sh "$pod:/tmp/log.sh"
  kubectl exec -t "$pod" -- bash /tmp/log.sh
  kubectl cp "$pod:/tmp/foo.log" "foo-$pod.log"
done

Writing autologging.sh


In [58]:
%%bash
chmod 777 ./autologging.sh && ./autologging.sh

tar: Removing leading `/' from member names
tar: Removing leading `/' from member names


In [59]:
!ls -la foo-*.log

-rw-rw-rw- 1 codespace codespace 3517 Sep 25 04:22 foo-nginx-deployment-59d4df587f-6w5c5.log
-rw-rw-rw- 1 codespace codespace 3552 Sep 25 04:22 foo-nginx-deployment-59d4df587f-gvxvl.log
-rw-rw-rw- 1 codespace codespace 3616 Sep 25 04:22 foo-nginx-deployment-59d4df587f-m8b7l.log


## Answers

## Teardown

In [60]:
%%bash
rm foo*.log
rm autologging.sh
kubectl delete pods --all
kubectl delete deployments --all
minikube stop

pod "nginx-deployment-59d4df587f-6w5c5" deleted
pod "nginx-deployment-59d4df587f-gvxvl" deleted
pod "nginx-deployment-59d4df587f-m8b7l" deleted
pod "nginx-pod" deleted
deployment.apps "nginx-deployment" deleted
* Stopping node "minikube"  ...
* Powering off "minikube" via SSH ...
* 1 node stopped.


## Submit

In [61]:
%%writefile backup.txt
# NOTE: For these commands your current working directory should be: 
# /workspaces/intro-to-kube/cyber-monday
# TODO: Enter kubectl cp command used to get `log.sh` file into the Pod
kubectl cp log.sh nginx-pod:/tmp/log.sh
# TODO: Enter kubectl exec command used to execute the `log.sh`
kubectl exec nginx-pod -- /bin/sh -c "chmod 777 /tmp/log.sh && touch /tmp/foo.log" && kubectl exec -t nginx-pod -- bash /tmp/log.sh
# TODO: Enter kubectl cp command used to copy the log file from inside the
# Pod to your current working directory
kubectl cp nginx-pod:/tmp/foo.log foo.log

Overwriting backup.txt


In [62]:
%%bash
git checkout -b cyber-ramnathv
git add nginx-deploy.yml nginx-pod.yml backup.txt
git commit -m "feat: project 1- cyber monday"
git push -u origin cyber-ramnathv
git checkout main

Switched to a new branch 'cyber-ramnathv'


[cyber-ramnathv 52ff341] feat: project 1- cyber monday
 3 files changed, 25 insertions(+), 9 deletions(-)


remote: 
remote: Create a pull request for 'cyber-ramnathv' on GitHub by visiting:        
remote:      https://github.com/ramnathv/intro-to-kube/pull/new/cyber-ramnathv        
remote: 
To https://github.com/ramnathv/intro-to-kube
 * [new branch]      cyber-ramnathv -> cyber-ramnathv


branch 'cyber-ramnathv' set up to track 'origin/cyber-ramnathv'.


Switched to branch 'main'


Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)
